### Design of Experiments
#### Hotelling's Weighin Problem

Weighing three objects using a pan scale over four measurements

In [ ]:
%matplotlib widget
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

# Settings
np.set_printoptions(precision=3)

# Helper functions
def setup_ax(_ax:plt.Axes):
    _ax.set_proj_type('ortho')
    _ax.set_aspect('equal')
    _ax.view_init(30, 30)
    _ax.set_xlim3d((-1,1))
    _ax.set_ylim3d((-1,1))
    _ax.set_zlim3d((-1,1))
    _ax.set_xticks((-1,0,1))
    _ax.set_yticks((-1,0,1))
    _ax.set_zticks((-1,0,1))
    _ax.tick_params('both', color=(1,1,1,0))
    _ax.set_xticklabels([])
    _ax.set_yticklabels([])
    _ax.set_zticklabels([])
    [_ax.quiver(0, 0, 0, *v, color=c, alpha=0.75) for v, c in zip(np.eye(3), ['r', 'g', 'b'])]
    [_ax.plot(*zip(p, p+d), '-k', linewidth=0.5)
     for o in product([-1, 0], repeat=3)
     for p in (np.array(np.meshgrid([0, 1], [0, 1], [0, 1])).T.reshape(-1, 3) + o)
     for d in np.eye(3) if all((p + d) <= np.array(o) + 1)]

In [ ]:
class HotellingExperiment(object):
    """
    A class for running the Hotelling's Weighing Experiment
    """
    def __init__(self, weights=3):
        np.random.seed(69)
        self.__weights = 25 * np.random.rand(weights,1)
        self.__design = None

    def set_design(self, design = np.zeros((4,3))):
        self.__design = design

    def run_experiment(self):
        if self.__design is None:
            print("A design must be assigned")
            return None, None
        measurements = np.dot(self.__design, self.__weights) + np.random.normal(0,1,(self.__design.shape[0],1))
        a_ij = np.linalg.inv(np.dot(self.__design.T, self.__design))
        estimates = np.dot(a_ij, np.dot(self.__design.T, measurements))
        return estimates
    
    def check_weights(self, measurement, std):
        measurement = np.array(measurement).reshape(self.__weights.shape)
        # Net difference
        net = np.linalg.norm(self.__weights - measurement, axis=1)
        # Mean + std difference
        upper = np.linalg.norm(self.__weights - (measurement + std), axis=1)
        # Mean - std difference
        lower = np.linalg.norm(self.__weights - (measurement - std), axis=1)
        return np.c_[net, upper, lower].T

In [ ]:
hotelling = HotellingExperiment()

design_list = []
design_name_list = []
# Naive approach
design = np.r_[np.zeros((1,3)), np.diag(np.ones(3))]
design_list.append(design)
design_name_list.append('Naive')

# Two-at-a-time approach
design = np.array([[0,0,0],
                   [1,1,0],
                   [0,1,1],
                   [1,0,1]])
design_list.append(design)
design_name_list.append('Two-at-a-time')

# Optimized approach 1
design = np.array([[ 1, 1, 1],
                   [-1, 1, 1],
                   [ 1,-1, 1],
                   [ 1, 1,-1]])
design_list.append(design)
design_name_list.append('Optimised 1')

# Optimized approach 2
design = np.array([[-1,-1,-1],
                   [-1, 1, 1],
                   [ 1,-1, 1],
                   [ 1, 1,-1]])
design_list.append(design)
design_name_list.append('Optimised 2')

# Iterate through designs
fig = plt.figure(figsize=(3*len(design_list),2*3))
estimates_per_design = []
for i, (design, design_name) in enumerate(zip(design_list, design_name_list)):
    # Plot approach configuration
    ax = fig.add_subplot(2, len(design_list), i+1, projection='3d')
    setup_ax(ax)
    ax.set_title(design_name)
    ax.plot(design[:,0], design[:,1], design[:,2], 'om', zorder=np.inf)

    # Simulate n experiments using current configuration
    hotelling.set_design(design=design)
    n_runs = 1000
    estimates_list = np.empty((n_runs,3))
    for j in range(n_runs):
        estimates = hotelling.run_experiment()
        estimates_list[j] = estimates.T
    # Plot measurement data
    estimates_per_design.append(estimates_list)
    ax = fig.add_subplot(2, len(design_list), i+1+len(design_list))
    ax.set_xlabel('Experiment Runs')
    ax.set_ylabel('Measured Weight' if i==0 else None)
    ax.plot(np.arange(n_runs)+1, estimates_list, linewidth=0.5)

In [ ]:
# Histograms per weight
fig = plt.figure(figsize=(9,3))
for j in range(estimates_per_design[0].shape[1]):
    ax = fig.add_subplot(1,3,j+1)
    ax.set_title("Weight " + str(j+1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.tick_params('both', color=(1,1,1,0))
    _, edges, _ = ax.hist(estimates_per_design[0][:,j], bins=20, color='m', label=design_name_list if j==0 else None)
    for i, (estimates, design_name) in enumerate(zip(estimates_per_design[1:], design_name_list[1:])):
        ax.hist(estimates[:,j], edges, label=design_name if j==0 else None)
fig.legend(loc='lower center')

In [ ]:
# Weight scatter
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot()
ax.set_aspect('equal')
ax.set_xlabel('Weight 1')
ax.set_ylabel('Weight 2')
for estimates, design_name in zip(estimates_per_design, design_name_list):
    ax.scatter(estimates[:,0], estimates[:,1], 2, alpha=0.5, label=design_name, zorder=np.inf)
fig.legend(loc='lower right')

In [ ]:
# Mean, Std. Variation, and Variance analysis
mnv_list = []
std_list = []
var_list = []
for estimates, design_name in zip(estimates_per_design, design_name_list):
    mean = np.mean(estimates, axis=0)
    std = np.std(estimates, axis=0)
    variance = np.var(estimates, axis=0)
    print(design_name)
    print("Mean:", mean, "Std. Deviation:", std, "Variance:", variance)
    var_list.append(std)
    std_list.append(variance)
    mnv_list.append(mean)
    mean_check = hotelling.check_weights(mean, std)
    print("Net diff.:", mean_check[0])
    print("Mean + std diff.:", mean_check[1])
    print("Mean - std diff.:", mean_check[2])

for var_other, design_name in zip(var_list[1:], design_name_list[1:]):
    print("Variance improvement by " + design_name + ":", var_list[0] / var_other)
for std_other, design_name in zip(std_list[1:], design_name_list[1:]):
    print("Std. Deviation improvement by " + design_name + ":", std_list[0] / std_other)